In [359]:
# 1. Import packages
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
print(keras.backend.backend())

tensorflow


In [360]:
from keras import layers,losses,metrics,activations,callbacks,initializers,regularizers,optimizers
import numpy as np
import pandas as pd
import tensorflow as tf
import os,datetime,pickle,sklearn
from sklearn import preprocessing,model_selection
from tensorflow.keras.models import Sequential
from tensorflow import keras
from sklearn import preprocessing, model_selection
import json

In [361]:
# 2. Data loading
CSV_PATH= os.path.join(os.getcwd(),'dataset','ecommerceDataset.csv')
df = pd.read_csv(CSV_PATH)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50424 entries, 0 to 50423
Data columns (total 2 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [362]:
# 3. Data inspection and Cleaning
print(df.columns)


Index(['Household', 'Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and s

In [363]:
print(df.head(1))

   Household  \
0  Household   

  Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for 

In [364]:
df.columns = ['category', 'text'] 
print(df['text'][0])
print(df['category'][0])
nClass = len(df['category'].unique())
print("Categories: ", df['category'].unique())
print("Number of categories: ", nClass)

SAF 'Floral' Framed Painting (Wood, 30 inch x 10 inch, Special Effect UV Print Textured, SAO297) Painting made up in synthetic frame with UV textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch (A perfect gift for your special ones).
Household
Categories:  ['Household' 'Books' 'Clothing & Accessories' 'Electronics']
Number of categories:  4


In [365]:
print("Missing Values in Dataset:")
print(df.isnull().sum())

Missing Values in Dataset:
category    0
text        1
dtype: int64


In [366]:
# Check for duplicate rows
print("\nNumber of Duplicate Rows:", df.duplicated().sum())


Number of Duplicate Rows: 22622


In [367]:
# Test by creating a new copy of the DF, then remove the duplicates
df_copy = df.copy()
df_copy = df_copy.drop_duplicates()
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27802 entries, 0 to 50409
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  27802 non-null  object
 1   text      27801 non-null  object
dtypes: object(2)
memory usage: 651.6+ KB


In [368]:
print("Label distribution from original:\n", df['category'].value_counts())
print("Label distribution after removing duplicates:\n", df_copy['category'].value_counts())

Label distribution from original:
 category
Household                 19312
Books                     11820
Electronics               10621
Clothing & Accessories     8671
Name: count, dtype: int64
Label distribution after removing duplicates:
 category
Household                 10563
Books                      6256
Clothing & Accessories     5675
Electronics                5308
Name: count, dtype: int64


In [369]:
# Drop null values
df.dropna(inplace=True)
# Drop duplicate rows
df.drop_duplicates(inplace=True)

In [370]:
# Check for duplicate rows
print("\nNumber of Duplicate Rows:", df.duplicated().sum())


Number of Duplicate Rows: 0


In [371]:
# 4. Split into feature and label
feature = df['text'].values
label = df['category'].values
# Encode the label
label_encoder = preprocessing.LabelEncoder()
label_encoded = label_encoder.fit_transform(label)
print(np.unique(label_encoded))
print(label_encoder.inverse_transform(np.unique(label_encoded)))

[0 1 2 3]
['Books' 'Clothing & Accessories' 'Electronics' 'Household']


In [372]:
# 5. Data splitting
seed = 42
x_train, x_split, y_train, y_split = model_selection.train_test_split(feature, label_encoded, train_size=0.7, random_state=seed)
x_val, x_test, y_val, y_test = model_selection.train_test_split(x_split, y_split, test_size=0.5, random_state=seed)

In [373]:
# 6. NLP processes
# (A) Tokenization
vocab_size = 10000
sequence_length = 200
tokenizer = layers.TextVectorization(max_tokens=vocab_size, output_sequence_length=sequence_length)
tokenizer.adapt(x_train)

In [374]:
# Test out the tokenizer
print(tokenizer.get_vocabulary())
sample_texts = x_train[:2]
sample_tokens = tokenizer(sample_texts)
print(sample_texts[0])
print(sample_tokens[0])

['', '[UNK]', np.str_('the'), np.str_('and'), np.str_('of'), np.str_('to'), np.str_('a'), np.str_('for'), np.str_('in'), np.str_('with'), np.str_('is'), np.str_('your'), np.str_('you'), np.str_('it'), np.str_('this'), np.str_('on'), np.str_('from'), np.str_('that'), np.str_('or'), np.str_('as'), np.str_('are'), np.str_('can'), np.str_('be'), np.str_('an'), np.str_('has'), np.str_('all'), np.str_('at'), np.str_('by'), np.str_('will'), np.str_('x'), np.str_('set'), np.str_('1'), np.str_('not'), np.str_('use'), np.str_('quality'), np.str_('made'), np.str_('book'), np.str_('its'), np.str_('which'), np.str_('have'), np.str_('product'), np.str_('easy'), np.str_('about'), np.str_('also'), np.str_('he'), np.str_('size'), np.str_('up'), np.str_('our'), np.str_('design'), np.str_('more'), np.str_('one'), np.str_('home'), np.str_('2'), np.str_('any'), np.str_('author'), np.str_('cotton'), np.str_('new'), np.str_('black'), np.str_('high'), np.str_('we'), np.str_('his'), np.str_('color'), np.str_('

In [375]:
# (B) Embedding
embedding_dim = 128
embedding = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)

In [376]:
# Test the embedding layer
sample_embedding = embedding(sample_tokens)
print(sample_embedding[0])

tf.Tensor(
[[-0.00771403 -0.04257231  0.01058847 ...  0.04908765  0.04160121
  -0.02772598]
 [ 0.00778275 -0.04565399 -0.01207691 ...  0.04785705 -0.01473466
  -0.02622285]
 [-0.01932525 -0.03397361  0.01137859 ...  0.00723753 -0.01577561
   0.01773492]
 ...
 [ 0.03214446 -0.00848166  0.00207651 ...  0.02454619  0.02439414
   0.02985745]
 [ 0.03214446 -0.00848166  0.00207651 ...  0.02454619  0.02439414
   0.02985745]
 [ 0.03214446 -0.00848166  0.00207651 ...  0.02454619  0.02439414
   0.02985745]], shape=(200, 128), dtype=float32)


In [377]:
# 7. Create the keras model
model = Sequential([
    tokenizer,
    embedding,
    layers.Bidirectional(layers.LSTM(32, return_sequences=True)),
    layers.Dropout(0.4),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

In [378]:
# 8. Compile the model
loss = losses.SparseCategoricalCrossentropy()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss=loss, metrics=['accuracy'])


In [379]:
# Define Early Stopping
early_stopping = callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-5)

In [380]:
# TensorBoard logging
time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
ts = callbacks.TensorBoard(log_dir=f"logs/fit{time}", histogram_freq=1)

In [381]:
# Train model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=32, epochs=10, callbacks=[ts, early_stopping])

Epoch 1/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 50s 74ms/step - accuracy: 0.4510 - loss: 1.2151 - val_accuracy: 0.8158 - val_loss: 0.4741
Epoch 2/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 48s 79ms/step - accuracy: 0.8570 - loss: 0.4476 - val_accuracy: 0.9374 - val_loss: 0.2649
Epoch 3/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 47s 78ms/step - accuracy: 0.9317 - loss: 0.2730 - val_accuracy: 0.9417 - val_loss: 0.2253
Epoch 4/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 48s 79ms/step - accuracy: 0.9524 - loss: 0.2077 - val_accuracy: 0.9408 - val_loss: 0.2180
Epoch 5/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 48s 79ms/step - accuracy: 0.9634 - loss: 0.1682 - val_accuracy: 0.9374 - val_loss: 0.2477
Epoch 6/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 49s 80ms/step - accuracy: 0.9722 - loss: 0.1386 - val_accuracy: 0.9386 - val_loss: 0.2358
Epoch 7/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 49s 80ms/step - accuracy: 0.9779 - loss: 0.1095 - val_accuracy: 0.9388 - val_loss: 0.2407
Epoch 8/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 48s 79ms/step - accuracy: 0.9708 - loss: 0.1363 - 

In [382]:
#save path
SAVE_DIR = os.path.join(os.getcwd(), "saved_models")
os.makedirs(SAVE_DIR, exist_ok=True)

model.save(os.path.join(SAVE_DIR, "lstm_model.keras"))

In [383]:
#Use the model to make prediction
y_pred = model.predict(x_test)
print(y_pred[0])
y_pred = np.argmax(y_pred,axis=1)
print(y_pred[0])
y_pred = label_encoder.inverse_transform(y_pred)
print(y_pred[0])

131/131 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step
[0.05507    0.00394499 0.90685135 0.03413364]
2
Electronics


In [384]:
from sklearn.metrics import classification_report

# Ensure y_pred is in integer form
y_pred_probs = model.predict(x_test)  # Get probabilities
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert to class labels (integers)

# Generate classification report
report = classification_report(y_test, y_pred)
print(report)


131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step
              precision    recall  f1-score   support

           0       0.96      0.91      0.93       941
           1       0.97      0.97      0.97       857
           2       0.85      0.91      0.88       777
           3       0.96      0.95      0.95      1596

    accuracy                           0.94      4171
   macro avg       0.93      0.94      0.93      4171
weighted avg       0.94      0.94      0.94      4171

